# IE 306  SYSTEM SIMULATION
## FURKAN KESKIN - 2018400150
## SINEM KOCOGLU - 2020400339

In [19]:
import numpy as np
from queue import Queue, PriorityQueue

SEED = 2018400150 + 2020400339

nurses = 4  # S
beds = 6    # K
p_1 = 0.25
lambd = 1
mu_T = 0.3125
mu_s = 0.16
mu_cb = 0.1666666667

## PART 1.2

In [20]:
iterations = 1500

def Generate_Interarrival():
    return np.random.exponential(1/lambd, iterations)

def Generate_Nurse_Service_Time():
    return np.random.exponential(1/mu_T, iterations)

def Generate_Hospital_Healing_Time():
    return np.random.exponential(1/mu_cb, iterations)

def Generate_Home_Healing_Time(condition):
    if condition == 's':
        return np.random.exponential(1/mu_s, iterations)
    elif condition == 'c':
        alfa = np.random.uniform(1.25, 1.75)
        return np.random.exponential(alfa/mu_cb, iterations)
    
def Generate_Conditions():
    condition_probs = np.random.uniform(0, 1, iterations)
    return np.array(['s' if i <= p_1 else 'c' for i in condition_probs])

In [21]:
np.random.seed(SEED)

interarrivals = Generate_Interarrival()
nurse_service_times = Generate_Nurse_Service_Time()
hospital_healing_times = Generate_Hospital_Healing_Time()
home_healing_times_stable = Generate_Home_Healing_Time('s')
home_healing_times_critical = Generate_Home_Healing_Time('c')
arrivals = np.roll(interarrivals.cumsum(), 2)
conditions = Generate_Conditions()

In [22]:
def Arrival():
    global busy_nurses, nurse_idx
    global total_number_of_people, patients_in_system
    
    total_number_of_people += 1
    patients_in_system += 1
    
    # adding next arrival to the FEL
    FEL.put((arrivals[pid-already_in_hospital+1], 'A', pid+1))
    
    if busy_nurses < nurses:
        FEL.put((simulation_time + nurse_service_times[nurse_idx], 'DN', pid))
        busy_nurses += 1
        nurse_idx += 1
    else:
        WQ.put((simulation_time, pid))
            
    

def Departure_Triage():
    global busy_nurses, occupied_beds, nurse_idx, condition_idx, hospital_idx, home_stable_idx, home_critical_idx, queue_wait_time
    global people_rejected, total_nurse_triage_time, patients_in_system
    
    # updating total nurse triage time
    total_nurse_triage_time += nurse_service_times[pid-initially_occupied_beds]
    
    # should be independent from the SEED
    # condition = 's' if np.random.uniform(0, 1) <= p_1 else 'c'
    condition = conditions[condition_idx]
    condition_idx += 1
    
    # departure of the current patient from the nurse triage
    if condition == 's':
        healing_time_dict[pid] = home_healing_times_stable[home_stable_idx]
        
        FEL.put((simulation_time + home_healing_times_stable[home_stable_idx], 'H', pid))
        home_stable_idx += 1
        patients_in_system -= 1
    elif condition == 'c':
        if occupied_beds < beds:
            healing_time_dict[pid] = hospital_healing_times[hospital_idx]
            
            FEL.put((simulation_time + hospital_healing_times[hospital_idx], 'DB', pid))
            occupied_beds += 1
            hospital_idx += 1
        else:
            healing_time_dict[pid] = home_healing_times_critical[home_critical_idx]
            
            FEL.put((simulation_time + home_healing_times_critical[home_critical_idx], 'H', pid))
            home_critical_idx += 1
            people_rejected += 1
            patients_in_system -= 1
            
    # arrangements for the FEL
    if WQ.empty():
        busy_nurses -= 1
        
    else:
        # arrival time queue'da ne kadar beklediğiyle alakalı
        arrival_time_of_incoming, pid_of_incoming = WQ.get()
        FEL.put((simulation_time + nurse_service_times[nurse_idx], 'DN', pid_of_incoming))
        queue_wait_time = simulation_time - arrival_time_of_incoming
        nurse_idx += 1
        
        

def Treated_at_Hospital():
    global occupied_beds, healed_patients
    global total_healing_time, patients_in_system 
    
    total_healing_time += healing_time_dict[pid]
     
    occupied_beds -= 1
    healed_patients += 1
    patients_in_system -= 1
    

def Healed_at_Home():
    global healed_patients, healed_at_home
    global total_healing_time 
    
    total_healing_time += healing_time_dict[pid]
    
    healed_at_home += 1
    healed_patients += 1
    
    
def Advance_Time():
    global simulation_time, event_type, pid, event_no
    simulation_time, event_type, pid = FEL.get()
    event_no += 1
    
    
def Execute_Event():
    global prev_simulation_time, empty_nurses_time, empty_beds_time, empty_nurses_and_beds_time, total_occupied_beds, queue_wait_time
    
    # setting queue wait time to 0 before execution of each event
    queue_wait_time = 0
    
    if busy_nurses != nurses:
        empty_nurses_time += (simulation_time - prev_simulation_time)
        
    if occupied_beds != beds:
        empty_beds_time += (simulation_time - prev_simulation_time)
        
    if busy_nurses != nurses and occupied_beds != beds:
        empty_nurses_and_beds_time += (simulation_time - prev_simulation_time)
    
    
    if event_type == 'A':
        Arrival()
    elif event_type == 'DN':
        Departure_Triage()
    elif event_type == 'DB':
        Treated_at_Hospital()
    elif event_type == 'H':
        Healed_at_Home()
        
           
    prev_simulation_time = simulation_time
    total_occupied_beds += occupied_beds
        
    # printing the simulation time, total number of sick people, number in the hospital and other necessary model outputs
    if event_no <= 50:
        print_table()
        
    Advance_Time()
 

## PART 2.2

In [23]:
# functions to print table and headers
def print_table():
    FEL_list = list(FEL.queue)
    x, y, z = FEL_list[0]        
    qwt = round(queue_wait_time, 3) if queue_wait_time != 0 else "-"
        
    print ("{:<8} {:<22} {:<24} {:<14} {:<14} {:<8} {:<11} {:<9} {:<2}".format(round(simulation_time, 3), str((round(x, 3), y, z)), str((round(x, 3), y, z)), busy_nurses, occupied_beds, WQ.qsize(), qwt, healed_patients, patients_in_system))
    for i in range(1, len(FEL_list)):
        x, y, z = FEL_list[i]
        print ("{:<8} {:<2}".format(" ", str((round(x, 3), y, z))))
    print("-----------------------------------------------------------------------------------------------------------------------------")
    
def print_headers():
    print ("{:<14} {:<18} {:<18} {:<14} {:<18} {:<8} {:<8} {:<10} {:<2}".format('Time','FEL','Next Event','Busy Nurses', 'Occupied Beds', 'WL', 'WQ', 'Healed', 'Sick'))
    print("-----------------------------------------------------------------------------------------------------------------------------")

In [24]:
print("WL : Length of waiting queue")
print("WQ : Waiting time in queue")
print("Healed : Total number of healed patients")
print("Sick : Total number of sick people = total number of patients in the system")
print("Busy Nurses : Current number of busy nurses")
print("Occupied Beds : Current number of occupied beds")

for termination_limit in ((20),(200),(1000)):
    for starting_condition in ((0, 0), (nurses//2, beds//2), (nurses, beds)):
        FEL = PriorityQueue()              # Future Event List  
        WQ = Queue()                       # Waiting Queue for Nurse Service
        healing_time_dict = {}             # pid: hospital_healing_times[hospital_idx]
        queue_wait_time = 0                # 

        # initial set-up
        prev_simulation_time = 0
        simulation_time = 0
        event_type = 'A'
        pid = 1
        event_no = 1
        
        # global constants 
        initially_busy_nurses = starting_condition[0]
        initially_occupied_beds = starting_condition[1]
        already_in_hospital = initially_busy_nurses + initially_occupied_beds

        # global variables
        busy_nurses = starting_condition[0]   
        occupied_beds = starting_condition[1]

        # initial activity indices 
        nurse_idx = 0
        condition_idx = 0
        hospital_idx = 0
        home_stable_idx = 0
        home_critical_idx = 0

        healed_patients = 0

        # 0, 1, 2 .... idx-1

        # System starts with a patient with ID = 1 in simulation time = 0
        # t, event, ID
        
        # creating initial patients in nurse triage
        for _ in range(busy_nurses):
            FEL.put((nurse_service_times[nurse_idx], 'DN', pid))
            nurse_idx += 1
            pid += 1
            
        # creating initial patients in hospital beds
        for _ in range(occupied_beds):
            healing_time_dict[pid] = hospital_healing_times[hospital_idx]
            
            FEL.put((hospital_healing_times[hospital_idx], 'DB', pid))
            hospital_idx += 1
            pid += 1
            
        # statistics
        total_number_of_people = already_in_hospital
        empty_nurses_time = 0
        empty_beds_time = 0
        empty_nurses_and_beds_time = 0
        people_rejected = 0
        total_nurse_triage_time = 0
        total_occupied_beds = 0
        healed_at_home = 0
        total_healing_time = 0
        patients_in_system = already_in_hospital

        # Printing tables for 3 condition and 3 different number of patients
        print()
        print()
        print("Number of Healed Patients:{}".format(termination_limit))
        print()
        if starting_condition==(0,0):
            print("* Empty System")
        elif starting_condition==(nurses,beds):
            print("* All nurses and beds full")
        else:
            print("* Half of the nurses and half of the beds full")
        print()
        print_headers()
        while healed_patients < termination_limit:
            Execute_Event()
            
        
        #Print statistics
        print()
        
        # long run marginal probability of being empty for triage or for the beds
        print((empty_nurses_time + empty_beds_time - empty_nurses_and_beds_time) / simulation_time)
        
        #  the joint probability of both being empty
        print(empty_nurses_and_beds_time / simulation_time)
        
        # average number of people rejected by bed area
        print(people_rejected / total_number_of_people)
        
        # average utilization of each triage nurse
        print(total_nurse_triage_time / (nurses*simulation_time))
        
        # average number of occupied beds in the hospital
        print(total_occupied_beds / (beds*event_no))
        
        # average number of patients that are treated at home
        print(healed_at_home / total_number_of_people)
        
        # average time a sick person gets better.
        print(total_healing_time / healed_patients)
        
        print("-----------------------------------------------------------------")
        
        



WL : Length of waiting queue
WQ : Waiting time in queue
Healed : Total number of healed patients
Sick : Total number of sick people = total number of patients in the system
Busy Nurses : Current number of busy nurses
Occupied Beds : Current number of occupied beds


Number of Healed Patients:20

* Empty System

Time           FEL                Next Event         Busy Nurses    Occupied Beds      WL       WQ       Healed     Sick
-----------------------------------------------------------------------------------------------------------------------------
0        (0.241, 'A', 2)        (0.241, 'A', 2)          1              0              0        -           0         1 
         (2.442, 'DN', 1)
-----------------------------------------------------------------------------------------------------------------------------
0.241    (0.888, 'A', 3)        (0.888, 'A', 3)          2              0              0        -           0         2 
         (2.442, 'DN', 1)
         (4.11, 'DN'